In [101]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import requests

load_dotenv()

CHAT_MODEL = 'gpt-4o-mini'
print('ready', openai_key[:10] + "**", CHAT_MODEL)

ready sk-proj-XV** gpt-4o-mini


In [102]:
class PostGenerator:


    def __init__(self, chat_model=CHAT_MODEL):
        openai_key = os.environ['OPENAI_API_KEY']
        client = OpenAI(
            api_key=openai_key,  # this is also the default, it can be omitted
        )
        self.client = client
        self.chat_model = chat_model
        self.max_tokens = 160

    # https://platform.openai.com/docs/guides/text-generation#text-generation-models
    def create_response_for_messages(self, messages):
        response = self.client.chat.completions.create(
            model=self.chat_model,
            messages=messages,
            max_tokens=self.max_tokens
        )
        print('response', response)
        return response.choices[0].message.content


    def generate_tweet(self, user_background, user_interests, target_audience, current_events):
        events_str = ', '.join(current_events)
        prompt = f"Generate a formatted tweet (with newlines) for someone with a background in {user_background}, who loves {user_interests}, targeting {target_audience}. Current events: {events_str}. Pick one of these events for the post and then take a hard stance or side on the topic with a short explanation as to why. No emojis or hashtags."
        print('generate_tweet', prompt)
        messages = [
            {'role': 'developer', 'content': 'You are a social media manager responsible for making posts. Do not make these posts sound robotics. Insert abbreviations and language to make the post sound human.'},
            {'role': 'user', 'content': prompt}
        ]
        tweet = self.create_response_for_messages(messages)
        return tweet


def format_article(a):
    text = f"{a['title']} {a["description"]}"
    return text.replace('[Removed]', '')

def get_current_events(theme):
    response = requests.get('https://newsapi.org/v2/everything', params={
        'apiKey': os.getenv('NEWS_API_KEY'),
        'q': theme,
        'pageSize': 5
    })
    articles = response.json().get('articles', [])
    articles = [format_article(a) for a in articles]
    return articles

current_events = None

In [103]:
# Example usage
background = "software, engineering, science, business. Worked at a startup three years as a head of engineering and at larger companies as a software engineer. "
interests = "coding, open-source projects, hackathons, and startups"
target_audience = "developers and entrepreneurs"
theme = "technology, science, controversial"

if not current_events:
  current_events = get_current_events(theme)
  print('fetched events', current_events)
else:
  print('using cached events', current_events)

fetched events ["Can RFK Jr make America's diet healthy again? Kennedy's plans to ban dyes in cereals and fluoride in water could get major pushback from the food industry.", 'Biden backs calls to ban congressional stock trading President Joe Biden lent his voice to the growing bipartisan calls on the Hill to end stock trading for congressional members.', 'The Download: words of wisdom from the departing White House tech advisor, and controversial AI manga translation This is today’s edition of\xa0The Download,\xa0our weekday newsletter that provides a daily dose of what’s going on in the world of technology. What the departing White House chief tech advisor has to say on AI President Biden’s administration will end within two m…', 'China sends cloud powered by homebrew Loongson CPUs into space Plus: Korea cracks down on Temu; US, Vietnam, sign infosec pact; India extends controversial hardware import law; and more\nAsia In Brief Chinese chip designer Loongson last Friday announced its

In [104]:
post_generator = PostGenerator()
tweet = post_generator.generate_tweet(background, interests, target_audience, current_events)


generate_tweet Generate a formatted tweet (with newlines) for someone with a background in software, engineering, science, business. Worked at a startup three years as a head of engineering and at larger companies as a software engineer. , who loves coding, open-source projects, hackathons, and startups, targeting developers and entrepreneurs. Current events: Can RFK Jr make America's diet healthy again? Kennedy's plans to ban dyes in cereals and fluoride in water could get major pushback from the food industry., Biden backs calls to ban congressional stock trading President Joe Biden lent his voice to the growing bipartisan calls on the Hill to end stock trading for congressional members., The Download: words of wisdom from the departing White House tech advisor, and controversial AI manga translation This is today’s edition of The Download, our weekday newsletter that provides a daily dose of what’s going on in the world of technology. What the departing White House chief tech adviso

In [105]:
print(tweet)


As a dev and startup enthusiast, I find the push from Biden to ban congressional stock trading crucial. Transparency is key in rebuilding trust in our political system. When lawmakers profit off stocks, it raises questions on their decision-making process. We need leaders focused on the greater good, not personal gains. This step could pave the way for more ethical governance, fostering a healthier environment for innovation and collaboration in tech and business. Let's prioritize integrity!
